## Пример c CartPole DQN

Сеть Deep Q = DQN

Cartpole - известный также как перевернутый маятник с центром тяжести над своей точкой поворота. Он нестабилен, но его можно контролировать, перемещая точку поворота под центром массы. Цель состоит в том, чтобы сохранить равновесие, прикладывая соответствующие усилия к точке поворота.

Другой env можно использовать без каких-либо изменений кода. Пространство состояний должно быть единым вектором, действия должны быть дискретными.

CartPole - самый простой. На ее решение должно уйти несколько минут.

Для LunarLander может потребоваться 1-2 часа, чтобы получить 200 баллов (хороший балл) в Colab, и прогресс в обучении не выглядит информативным.


Код в Задании: https://colab.research.google.com/drive/1mvYm-rkEEADXtkYH0qKr8c-A9fjlc6Sm#scrollTo=wSCj4g0whSsr


In [ ]:
import os
################################################
# For CartPole
################################################

!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/atari_wrappers.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/utils.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/replay_buffer.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/framebuffer.py

!pip install gym[box2d]

!touch .setup_complete


# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [ ]:
import gym
ENV_NAME = 'CartPole-v1'

def make_env(seed=None):
    # some envs are wrapped with a time limit wrapper by default
    env = gym.make(ENV_NAME).unwrapped
    if seed is not None:
        env.seed(seed)
    return env

In [ ]:
import matplotlib.pyplot as plt
env = make_env()
env.reset()
plt.imshow(env.render("rgb_array"))
state_shape, n_actions = env.observation_space.shape, env.action_space.n

### Building a network

Теперь нам нужно построить нейронную сеть, которая может сопоставлять наблюдения с состоянием q-значений. Модель еще не должна быть слишком сложной. 1-2 скрытых слоя с < 200 нейронами и активацией ReLU, вероятно, будет достаточно. Batch normalization и dropout могут все испортить, поэтому их не используем


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# those who have a GPU but feel unfair to use it can uncomment:
# device = torch.device('cpu')
device
print(state_shape)

In [ ]:
from torch import nn
class DQNAgent(nn.Module):
    def __init__(self, state_shape, n_actions, epsilon=0):

        super().__init__()
        self.epsilon = epsilon
        self.n_actions = n_actions
        self.state_shape = state_shape
        # Define your network body here. Please make sure agent is fully contained here
        assert len(state_shape) == 1
        state_dim = state_shape[0]
        
        
        # Define NN
        ##############################################
        hidden_size = 150
        self._nn = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
            nn.ReLU()
        )
        ##############################################

        
    def forward(self, state_t):
        """
        takes agent's observation (tensor), returns qvalues (tensor)
        :param state_t: a batch states, shape = [batch_size, *state_dim=4]
        """
        # Use your network to compute qvalues for given state
        
        ##############################################
        qvalues = self._nn(state_t)
        ##############################################

        assert qvalues.requires_grad, "qvalues must be a torch tensor with grad"
        assert (
            len(qvalues.shape) == 2 and 
            qvalues.shape[0] == state_t.shape[0] and 
            qvalues.shape[1] == n_actions
        )

        return qvalues

    def get_qvalues(self, states):
        """
        like forward, but works on numpy arrays, not tensors
        """
        model_device = next(self.parameters()).device
        states = torch.tensor(states, device=model_device, dtype=torch.float32)
        qvalues = self.forward(states)
        return qvalues.data.cpu().numpy()

    def sample_actions(self, qvalues):
        """pick actions given qvalues. Uses epsilon-greedy exploration strategy. """
        epsilon = self.epsilon
        batch_size, n_actions = qvalues.shape

        random_actions = np.random.choice(n_actions, size=batch_size)
        best_actions = qvalues.argmax(axis=-1)

        should_explore = np.random.choice(
            [0, 1], batch_size, p=[1-epsilon, epsilon])
        return np.where(should_explore, random_actions, best_actions)

In [ ]:
agent = DQNAgent(state_shape, n_actions, epsilon=0.5).to(device)

In [ ]:
def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """ Plays n_games full games. If greedy, picks actions as argmax(qvalues). Returns mean reward. """
    rewards = []
    for _ in range(n_games):
        s = env.reset()
        reward = 0
        for _ in range(t_max):
            qvalues = agent.get_qvalues([s])
            action = qvalues.argmax(axis=-1)[0] if greedy else agent.sample_actions(qvalues)[0]
            s, r, done, _ = env.step(action)
            reward += r
            if done:
                break

        rewards.append(reward)
    return np.mean(rewards)

### Experience Replay Buffer and Target Networks

#### Интерфейс довольно прост:
* `exp_replay.add(obs, act, rw, next_obs, done)` - сохраняет (s,a,r,s',done) кортеж в буффер
* `exp_replay.sample(batch_size)` - возвращает observations, actions, rewards, next_observations и is_done для `batch_size` random samples.
* `len(exp_replay)` - возвращает количество элементов хранящихся в replay buffer.

In [ ]:
from replay_buffer import ReplayBuffer

exp_replay = ReplayBuffer(2000)


target_network = DQNAgent(agent.state_shape, agent.n_actions, epsilon=0.5).to(device)
# This is how you can load weights from agent into target network
target_network.load_state_dict(agent.state_dict())

### TD-Loss

Вычислим ошибку TD Q-learning:

$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

С Q-reference определенным как

$$ Q_{reference}(s,a) = r(s,a) + \gamma \cdot max_{a'} Q_{target}(s', a') $$

Где
* $Q_{target}(s',a')$ обозначает $Q$-значение следующего предсказанного состояния и следующего действия  __target_network__
* $s, a, r, s'$ текущее состояние, действие, вознаграждение и следующее состояние соответственно
* $\gamma$ является коэффициентом дисконтирования, определенным двумя ячейками выше.

In [ ]:
def compute_td_loss(states, actions, rewards, next_states, is_done,
                    agent, target_network,
                    gamma=0.99,
                    check_shapes=False,
                    device=device):
    """ Compute td loss using torch operations only. Use the formulae above. """
    states = torch.tensor(states, device=device, dtype=torch.float32)    # shape: [batch_size, *state_shape]
    actions = torch.tensor(actions, device=device, dtype=torch.int64)    # shape: [batch_size]
    rewards = torch.tensor(rewards, device=device, dtype=torch.float32)  # shape: [batch_size]
    # shape: [batch_size, *state_shape]
    next_states = torch.tensor(next_states, device=device, dtype=torch.float)
    is_done = torch.tensor(
        is_done.astype('float32'),
        device=device,
        dtype=torch.float32,
    )  # shape: [batch_size]
    is_not_done = 1 - is_done

    # get q-values for all actions in current states
    predicted_qvalues = agent(states)  # shape: [batch_size, n_actions]

    # compute q-values for all actions in next states
    # with torch.no_grad():
    predicted_next_qvalues = target_network(next_states)  # shape: [batch_size, n_actions]
    
    # select q-values for chosen actions
    predicted_qvalues_for_actions = predicted_qvalues[range(len(actions)), actions]  # shape: [batch_size]

    # compute V*(next_states) using predicted next q-values
    ##############################################
    next_state_values = predicted_next_qvalues.max(axis=-1)[0]
    ##############################################

    assert next_state_values.dim() == 1 and next_state_values.shape[0] == states.shape[0], \
        "must predict one value per state"

    # compute "target q-values" for loss - it's what's inside square parentheses in the above formula.
    # at the last state use the simplified formula: Q(s,a) = r(s,a) since s' doesn't exist
    # you can multiply next state values by is_not_done to achieve this.
    ###############################################
    target_qvalues_for_actions = rewards + gamma * next_state_values * is_not_done
    ##############################################

    # mean squared error loss to minimize
    loss = torch.mean((predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2)

    if check_shapes:
        assert predicted_next_qvalues.data.dim() == 2, \
            "make sure you predicted q-values for all actions in next state"
        assert next_state_values.data.dim() == 1, \
            "make sure you computed V(s') as maximum over just the actions axis and not all axes"
        assert target_qvalues_for_actions.data.dim() == 1, \
            "there's something wrong with target q-values, they must be a vector"

    return loss

### Main loop

In [ ]:
import random

# your favourite random seed
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

env = make_env(seed)
state_dim = env.observation_space.shape
n_actions = env.action_space.n
state = env.reset()

agent = DQNAgent(state_dim, n_actions, epsilon=1).to(device)
target_network = DQNAgent(state_dim, n_actions, epsilon=1).to(device)
target_network.load_state_dict(agent.state_dict())

Один агент для игры, второй для обучения. Периодически веса обученного агента копируются в "игрока"

In [ ]:
def play_and_record(initial_state, agent, env, exp_replay, n_steps=1):
    """
    Play the game for exactly n_steps, record every (s,a,r,s', done) to replay buffer. 
    Whenever game ends, add record with done=True and reset the game.
    It is guaranteed that env has done=False when passed to this function.

    PLEASE DO NOT RESET ENV UNLESS IT IS "DONE"

    :returns: return sum of rewards over time and the state in which the env stays
    
    hint: use agent.sample.actions
    """
    s = initial_state
    sum_rewards = 0

    # Play the game for n_steps as per instructions above
    # <YOUR CODE>
    for _ in range(n_steps):
        qvalues = agent.get_qvalues([s])
        
        action = agent.sample_actions(qvalues)[0]
        # action = action.argmax(axis=-1)[0]
        state, reward, done, _ = env.step(action)
        sum_rewards += reward
        
        exp_replay.add(s, action, reward, state, done)
        
        if done:
            state = env.reset()
        
        s = state

    return sum_rewards, s

In [ ]:
# import src.L15_RL.lib.utils
import utils
from warnings import simplefilter

simplefilter("ignore", category=UserWarning)

REPLAY_BUFFER_SIZE = 10**4

exp_replay = ReplayBuffer(REPLAY_BUFFER_SIZE)
for i in range(100):
    if not utils.is_enough_ram(min_available_gb=0.1):
        print("""
            Less than 100 Mb RAM available. 
            Make sure the buffer size in not too huge.
            Also check, maybe other processes consume RAM heavily.
            """
             )
        break
    play_and_record(state, agent, env, exp_replay, n_steps=10**2)
    if len(exp_replay) == REPLAY_BUFFER_SIZE:
        break
print(len(exp_replay))

In [ ]:
import time

timesteps_per_epoch = 1
batch_size = 32
total_steps = 4 * 10**4
decay_steps = 1 * 10**4

optimizer = torch.optim.Adam(agent.parameters(), lr=1e-4)

init_epsilon = 1
final_epsilon = 0.1

loss_freq = 20
refresh_target_network_freq = 100
eval_freq = 1000

max_grad_norm = 5000

mean_rw_history = []
td_loss_history = []
grad_norm_history = []
initial_state_v_history = []
step = 0

def wait_for_keyboard_interrupt():
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        pass

In [ ]:
from tqdm import trange
from IPython.display import clear_output

state = env.reset()
with trange(step, total_steps + 1) as progress_bar:
    for step in progress_bar:
        if not utils.is_enough_ram():
            print('less that 100 Mb RAM available, freezing')
            print('make sure everything is ok and use KeyboardInterrupt to continue')
            wait_for_keyboard_interrupt()

        agent.epsilon = utils.linear_decay(init_epsilon, final_epsilon, step, decay_steps)

        # play
        _, state = play_and_record(state, agent, env, exp_replay, timesteps_per_epoch)

        # train
        # <YOUR CODE: sample batch_size of data from experience replay>
        s,a,r,next_s, is_done = exp_replay.sample(batch_size)
        # loss = <YOUR CODE: compute TD loss>
        loss = compute_td_loss(s, a, r, next_s, is_done, agent, target_network)

        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()

        if step % loss_freq == 0:
            td_loss_history.append(loss.data.cpu().item())
            grad_norm_history.append(grad_norm.data.cpu().item())

        if step % refresh_target_network_freq == 0:
            # Load agent weights into target_network
            # <YOUR CODE>
            target_network.load_state_dict(agent.state_dict())

        if step % eval_freq == 0:
            mean_rw_history.append(evaluate(
                make_env(seed=step), agent, n_games=3, greedy=True, t_max=1000)
            )
            initial_state_q_values = agent.get_qvalues(
                [make_env(seed=step).reset()]
            )
            initial_state_v_history.append(np.max(initial_state_q_values))

            clear_output(True)
            print("buffer size = %i, epsilon = %.5f" %
                (len(exp_replay), agent.epsilon))

            plt.figure(figsize=[16, 9])

            plt.subplot(2, 2, 1)
            plt.title("Mean reward per episode")
            plt.plot(mean_rw_history)
            plt.grid()

            assert not np.isnan(td_loss_history[-1])
            plt.subplot(2, 2, 2)
            plt.title("TD loss history (smoothened)")
            plt.plot(utils.smoothen(td_loss_history))
            plt.grid()

            plt.subplot(2, 2, 3)
            plt.title("Initial state V")
            plt.plot(initial_state_v_history)
            plt.grid()

            plt.subplot(2, 2, 4)
            plt.title("Grad norm history (smoothened)")
            plt.plot(utils.smoothen(grad_norm_history))
            plt.grid()

            plt.show()

In [ ]:
final_score = evaluate(
  make_env(),
  agent, n_games=30, greedy=True, t_max=1000
)
print('final score:', final_score)
if final_score > 300:
  print('Well done')
else:
  print('not good enough for DQN')

## Дальнейшие идеи



<table><tr>
<td> <img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L15/problem_of_overestimation.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L15/problem_of_overestimation_diagramm.png" alt="Drawing" /> </td>
</tr></table>

Допустим, что $Q(s,a)$ распределен случайно. То есть, выбор action никак не влияет. Но так-как мы берем max по $Q$, то будет казаться, что мы можем получать большие значения

**Проблема переоценки**

Мы используем оператор "max" для вычисления цели
$$
L(s, a)=\left(Q(s, a)-\left(r+\gamma \max _{a^{\prime}} Q\left(s^{\prime}, a^{\prime}\right)\right)\right)^{2}
$$

**И новая проблема**

(тк мы хотим, чтобы  $\mathrm{E}_{s \sim S, a \sim A}[L(s, a)]$ было равно нулю)

### Другие улучшения DQN

**Prioritized Experience Replay**

Минибатчи из памяти выбираются не с равномерным распределением, а добавляем туда больше примеров, в которых предсказанные значения $Q$ сильнее всего отличаются от корректных. Т.е. примеры с максимальным **TD error** получают максимальный приоритет.

**Dueling networks**

Основная идея в том, что мы разделяем нашу сеть на две головы, одна из которых предсказывает абсолютное значение состояния \\( V(S) \\), а вторая - относительное преимущество одних действий над другими \\( A(s, a) = Q(s, a) - V(s) \\). Это преимущество называется advantage. Далее из этих двух значений мы собираем нашу Q-функцию, как \\( Q(s,a) = V(s) + A(a) \\)

**Noisy nets**

Т.к. по мере обучения агент будет стремиться выбирать состояния с максимальным $Q$, среди уже исследованных, это может помешать ему найти более эффективные состояния, в которых его ещё не было. Одним из решений этой проблемы является использование детерминированной и случайной нейросети, распределение параметров которой так же обучается с помощью градиентного спуска.

**Multi-step learning/n-step learning**

Основная идея в том, чтобы считать функцию ценности не по двум соседним примерам, а сразу по n. Это позволяет сети лучше запоминать длинные последовательности действий.

**Distributional RL**

Детерминированное значение $Q$ заменяется случайным распределением $Z$ с некоторыми параметрами, которые определяются в ходе обучения.

**Rainbow**

State of the art в развитии Q-обучения - набор перечисленных выше твиков. На графике ниже сравнение различных алгоритмов по количеству очков, усреднённое по играм Atari в сравнении со средними результатами человека.

<img src="https://edunet.kea.su/repo/EduNet-content/L15/img_license/rainbow_dqn_compare_different_algorithm.png" width="650">

### Double DQN


**Double Q-learning (NIPS 2010)**

$y=r+\gamma \max _{a^{\prime}} Q\left(s^{\prime}, a^{\prime}\right)$
- Q-learning target

$y=r+\gamma Q\left(s^{\prime}, \operatorname{argmax}_{a^{\prime}} Q\left(s^{\prime}, a^{\prime}\right)\right)$
- Rewritten $Q$-learning target

**Idea**: используем две оценки q-values: $Q^{A}, Q^{B}$
Они должны компенсировать ошибки друг друга, потому что они будут независимыми. Получим argmax от другой оценки

$y=r+\gamma Q^{A}\left(s^{\prime}, \operatorname{argmax}_{a} Q^{B}\left(s^{\prime}, a^{\prime}\right)\right)$ - Double $\mathrm{Q}$-learning target

То есть теперь есть две Q функции, которые мы обучаем: $Q^{A}$ и $Q ^{\text {B}}$ если ошибки независимы, то одна Q функция скажет, где выпал шум (где значения выше), а вторая тогда попытается взять значение, которое больше. И тогда
у нее не будет той же ошибки. Значение будет другое. $\Rightarrow$ Надеемся,
что проблема будет частично решена.

# Альтернативные подходы


**Actor-CriticI**: Обучить актера, который предсказывает действия(policy gradient) и критика, который предсказывает будущие награды от предсказанных действий(Q-Learning) 

"Asynchronous Methods for Deep Reinforcement Learning", ICML 2016

**Model-Based**: Обучить модель переходу состояния  $P\left(s_{t+1} \mid s_{t}, a_{t}\right)$  и затем использовать модель для принятия решений 

**Imitation Learning**: Собрать данные о том, как эксперты работают в окружающей среде, обучить функцию, чтобы имитировать то, что они делают (подход к обучению под наблюдением)


**Inverse Reinforcement Learning**:
Соберите данные экспертов, работающих в среде, изучите функцию вознаграждения, которую они, по-видимому, оптимизируют, затем используйте RL для этой функции вознаграждения.

"Algorithms for Inverse Reinforcement Learning", ICML 2000

**Adversarial Learning**: Научиться обманывать дискриминатор, который классифицирует действия, как верные/ошибочные

Ho and Ermon, "Genertive Adversaraal limitation Leaming", NeurlPS 2016

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L15/exploration_over_time.png" alt="Drawing" /> 

Чему будет равняться Q функция в клетках?

<img src="https://edunet.kea.su/repo/EduNet-content/L15/img_license/q_learning_vs_sarsa.png" alt="Drawing" /> 



Алгоритм SARSA считает более реалистичные оценки. Так-как учитывает прыжки робота в лаву, то есть считает оценку по $\epsilon -greedy$ политике.